```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

## Figure maker for comparing schemes

This notebook is for creating informative figures for comparing different oceanographic properties between numerical schemes.
All data is expected to be read in from NetCDF files.



#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

import os
import pyopencl
import datetime
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our Simulation Reader
from SWESimulators import SimReader


## Linear Coriolis

In this case, slow developing effects caused by non-constant Coriolis forces are considered.
Let the Coriolis parameter be defined as 
$$f(y) = \tilde{f} + \beta y,$$
on a region with flat bottom topography.
Consider then an initial state similar to in the case of Rossby adjustment (ref to equation), and let the boundary be open, so that initial energy is allowed to leave the domain.
Instead of the steady state obtained in Section XX, the varying Coriolis will produce slowgoing NAME OF WAVES waves travelling westwards.

Capturing slow-developing effects by numerical methods are sometimes challenging, as derivatives have a tendency to be smoothed by certain schemes. 
This case will therefore test how well the selected numerical schemes can handle such waves.


In [ ]:
# Define file names
folder = "netcdf_2017_12_07/"
fbl_file_name =   folder+"FBL_2017_12_07-16_08_12.nc"
ctcs_file_name =  folder+"CTCS_2017_12_07-16_09_52.nc"
kp07_file_name =  folder+"KP07_2017_12_07-16_11_07.nc"
cdklm_file_name = folder+"CDKLM16_2017_12_07-16_10_52.nc"

In [ ]:
reload(SimReader)
fbl_sim = SimReader.SimNetCDFReader(fbl_file_name)
fbl_sim.printAttributes()